In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

from IPython.display import clear_output

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"
pressure_file ="./data/pressure.csv"
wholerock_file = "./data/wholerock.csv"

melt = mp.read_melt(melt_file, index_col=["name"], units="wt. %")
olivine = mp.read_olivine(olivine_file, index_col=["name"], units="wt. %")
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

In [3]:
x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)
FeO_model = FeOi_predict.model

Here we test Kd models, but you can also do this for Fe3Fe2 or melt thermometers with `mpc.Fe3Fe2_models` or `mpc.melt_thermometers`

In [4]:
Kd_models = mpc.Kd_ol_FeMg_models
results = pd.DataFrame(index=melt.index, dtype=float)

for i, model in enumerate(Kd_models):

    mpc.model_configuration.Kd_model = model
    clear_output()
    print(f"model: {model}\n{i+1:03}/{len(Kd_models):03}")

    pec_model = mpc.PEC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeO_model)
    melts_corrected, pec, checks = pec_model.correct()

    results[model] = pec["total_crystallisation"]
    

model: toplis2005
008/008
Equilibrating ... |██████████████████████████████| 100% [10/10] in 4.2s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 14.7s                          | 0% [0/10] in 0s 


In [5]:
results

,blundy2020,putirka2016_8a,putirka2016_8b,putirka2016_8c,putirka2016_8d,saper2022,sun2020,toplis2005
name,,,,,,,,
PI032-04-01,9.615161,10.931458,10.835327,10.940723,11.450806,6.979077,11.060718,10.001428
PI032-04-02,11.017993,12.268896,12.265845,12.265845,12.456689,8.184579,12.224097,11.355652
PI041-02-02,1.159460,2.129163,2.130688,2.327637,1.930688,-0.800439,1.762512,0.383655
PI041-03-01,14.807861,16.084595,15.498218,15.692114,14.516528,11.709106,17.995654,13.184424
PI041-03-03,14.552490,15.759033,15.256470,15.368188,13.874780,11.200000,17.656470,12.684692
PI041-05-04,-2.363037,-1.811646,-2.338623,-1.990283,-2.335571,-4.787354,-2.153882,-4.023315
PI041-05-06,3.688623,4.649170,4.102356,4.091675,3.516089,0.810519,4.502356,2.179736
PI041-07-01,13.548462,14.454077,13.841650,14.263232,12.951514,9.992810,15.403174,12.418433
PI041-07-02,12.960449,14.075220,13.496582,13.884375,12.571130,9.383655,15.361975,11.285132
